In [100]:

import cobra
import cobra.io
from IPython.display import HTML, display, SVG, Markdown, Image
import os, sys
import pandas as pd
from io import StringIO
import numpy as np
import toolz
import qgrid
from pylab import figure, xlabel, ylabel, title, savefig, gca
import matplotlib
from matplotlib.sankey import Sankey
from d3flux import flux_map

def interleave(lower,upper, lower_level, upper_level, level_name, nconditions):
    interleaved =  pd.concat( [ lower, upper], 
                              axis=1)\
                    [ list( toolz.interleave( [lower, 
                                     upper]))]
    interleaved.columns.set_levels( levels  = [lower_level, upper_level]*nconditions, 
                           level   =   level_name,
                           inplace =    True )
    return interleaved
        
def add_cofactor_ratio_to_log_Q( log_Q, cofactor_ratios, model, cofactor ):
    cofactor_mets = set([met.id for met in model.metabolites if met.name== cofactor])
    for rxn in model.reactions:
        if rxn.id in log_Q.index:
            if rxn_contains_reactant_p( rxn, cofactor_mets ):
                log_Q.loc[rxn.id] +=  np.log( cofactor_ratios )
            elif rxn_contains_product_p( rxn , cofactor_mets ):
                log_Q.loc[rxn.id] -=  np.log( cofactor_ratios )            
    return log_Q

def apply_2nd_law( model,deltaG, lower, upper, tol=0.1 ):
    """
    Given a cobra model and a change in gibbs free energy, set reaction direction appropriately
    """
    for rxn in model.reactions:
        if rxn.id in deltaG.index:
            x,y,z = deltaG.index.get_loc(rxn.id), deltaG.columns.get_loc(lower), deltaG.columns.get_loc(upper)
            if deltaG.iloc[ x,y] > tol:
                rxn.upper_bound = 0
            elif deltaG.iloc[x,z] < -tol:
                rxn.lower_bound = 0
    return model

def rxn_contains_metabolite_p( rxn, mets ):
    return len(set([m.id for m in rxn.metabolites]) & mets) > 0

def rxn_contains_reactant_p( rxn, mets ):
    return len(set([m.id for m in rxn.reactants]) & mets) > 0

def rxn_contains_product_p( rxn, mets ):
    return len(set([m.id for m in rxn.products]) & mets) > 0


def get_rxns_w_1_met( mets, cobra_model ):
    rxns = set()
    for met in mets:
        rxns |= set([rxn.id for rxn in cobra_model.metabolites.get_by_id( met ).reactions])
    return rxns

def get_rxns_w_all_mets( mets, cobra_model ):
    rxns = set()
    for rxn in cobra_model.reactions:
        if all([met.id in mets 
                    for met in rxn.metabolites]):
            rxns.add( rxn.id )    
    return rxns
def getS(cobra_model):
    return pd.DataFrame(cobra.util.array.create_stoichiometric_matrix(cobra_model), 
                    index=[m.id for m in cobra_model.metabolites],
                    columns = [r.id for r in cobra_model.reactions])

def get_mu0( cobra_model, std_chemical_potentials, mu0_col):
    mu0 = {}
    for met in cobra_model.metabolites:
        if 'kegg.compound' in met.annotation and met.annotation['kegg.compound'] in std_chemical_potentials.index:
            met.notes['mu0'] = std_chemical_potentials.loc[met.annotation['kegg.compound'],mu0_col]
            mu0[met.id] = met.notes['mu0']
    mu0 = pd.Series(mu0)
    return mu0, cobra_model

def translate_metabolites( filename ):
    met_trans = {} 
    un_trans = []
    with open( filename ) as met_file:
        i = 0
        for line in met_file:
            cols= line.split('\t')
            if cols[0] == 'success':
                met_trans[metabolites[i]] = dict([col.split(':')
                                       for col in cols[:-1]
                                           if ':' in col])
                met_trans[metabolites[i]]['InChI'] = cols[-1]
            else:
                un_trans.append(metabolites[i])
            i += 1
    return met_trans, un_trans
def explode_series(series):
    return pd.concat([pd.DataFrame(list(v), index=np.repeat(k,len(v))) 
            for k,v in series.to_dict().items()
              ])    
def explode_dataframe(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

def get_producers( met, fluxes ):
    producers = []
    production_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] < 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] > 0) and (met in rxn.products)):
            producers.append( rxn )
            production_flux[rxn.id] = abs(fluxes[rxn.id]*rxn.metabolites[met])
    return producers, pd.Series(production_flux)

def get_consumers( met, fluxes ):
    consumers = []
    consumption_flux = {}
    for rxn in met.reactions:
        if ((fluxes[rxn.id] > 0) and (met in rxn.reactants)) or ((fluxes[rxn.id] < 0) and (met in rxn.products)):
            consumers.append( rxn )
            consumption_flux[rxn.id] = abs(fluxes[rxn.id]*rxn.metabolites[met])
    return consumers, pd.Series(consumption_flux)

def build_kegg_rxn( rxn ):
    reactants, products = [],[]
    for met, coeff in rxn.metabolites.items():
        if coeff < 0:
            if coeff == -1:
                reactants.append( met.annotation['kegg.compound'])
            else:
                reactants.append( '{} {}'.format( np.abs(coeff), met.annotation['kegg.compound']))
        elif coeff == 1:
            products.append( met.annotation['kegg.compound'])
        else:
            products.append( '{} {}'.format( np.abs(coeff), met.annotation['kegg.compound']))
    kegg_eqn =  '{} = {}'.format(' + '.join( reactants ), ' + '.join( products ) )
    return kegg_eqn
                                 
def rxn_can_produce_met_p( rxn, met ):
    return (rxn.lower_bound < 0 and met in rxn.reactants) or (rxn.upper_bound > 0 and met in rxn.products)

def rxn_can_consume_met_p( rxn, met ):
    return (rxn.lower_bound < 0 and met in rxn.products) or (rxn.upper_bound > 0 and met in rxn.reactants)

def build_rxn_w_compartment( rxn, use_metabolite_names=True ):
    reactants, products = [],[]
    for met, coeff in rxn.metabolites.items():
        if use_metabolite_names:
            met_name_or_id = met.name
        else:
            met_name_or_id = met.id
        if coeff < 0:
            if coeff == -1:
                reactants.append('{}[{}]'.format( met_name_or_id, met.compartment))
            else:
                reactants.append( '{} {}[{}]'.format( np.abs(coeff), met_name_or_id, met.compartment))
        elif coeff == 1:
            products.append( '{}[{}]'.format(met_name_or_id,met.compartment))
        else:
            products.append( '{} {}[{}]'.format( np.abs(coeff), met_name_or_id, met.compartment))
    if rxn.lower_bound < 0 and rxn.upper_bound > 0:
        direction = '<=>'
    elif rxn.lower_bound < 0:
        direction = '<--'
    elif rxn.upper_bound > 0:
        direction = '-->'
    else:
        direction = '--'
    rxn_eqn =  '{} {} {}'.format(' + '.join( reactants ), direction, ' + '.join( products ) )
    return rxn_eqn
                         
        
def summarize( met, fluxes, out=sys.stdout, use_metabolite_names=True, tol = 1e-5 ):
    producers, production_flux = get_producers( met, fluxes )
    production_pct = production_flux/production_flux.sum()*100
    ptitle = '\nTotal Production flux: {} -- {}[{}] ({})\n'.format(production_flux.sum(),
                                                                   met.name, 
                                                                   met.compartment, 
                                                                   met.id)
    pcols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ptitle)
    out.write('-'*len(ptitle) + '\n')
    out.write(pcols)
    out.write('\t'.join(['-'*len(col) for col in pcols.split('\t')]) + '\n')
    
    for producer in producers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format( 
            production_pct[producer.id], 
            production_flux[producer.id], 
            producer.id, 
            build_rxn_w_compartment( producer, 
                                     use_metabolite_names=True ),
            build_rxn_w_compartment( producer, 
                                     use_metabolite_names=False)))
    consumers, consumption_flux = get_consumers( met, fluxes )
    consumption_pct = consumption_flux/consumption_flux.sum()*100
    ctitle = '\nTotal Consumption flux: {} -- {}[{}] ({})\n'.format(consumption_flux.sum(), 
                                                                    met.name, met.compartment, met.id)
    ccols = 'Pct\tFlux\tRxn ID\tReaction\tReactionIDs\n'
    out.write(ctitle)
    out.write('-'*len(ctitle) + '\n')
    out.write(ccols)
    out.write('\t'.join(['-'*len(col) for col in ccols.split('\t')]) + '\n')
    
    for consumer in consumers:
        out.write('{:0.3f}\t{:0.3f}\t{}\t{}\t{}\n'.format(consumption_pct[consumer.id], 
                                            consumption_flux[consumer.id], 
                                            consumer.id, 
                                            build_rxn_w_compartment(consumer, use_metabolite_names=True),
                                            build_rxn_w_compartment(consumer, use_metabolite_names=False)))
def get_label( rxn, id_only=False ):
    if id_only:
        return rxn.id
    else:
        return '{} ({})'.format( rxn.name, rxn.id)

def group_fluxes(rxns, fluxes, min_group_flux_pct, max_group_flux_pct,cull_p=False ):
    """fluxes less than min_group_flux_pct are grouped together in bins of max_group_flux_pct size to prevent clutter"""
    groups =[]
    group_labels = []
    group_flows = 0
    for rxn in rxns:
        if fluxes[rxn.id]/fluxes.sum()*100 >= min_group_flux_pct:
            groups.append( ( get_label( rxn ), fluxes[rxn.id] ) )
        elif not cull_p:
            group_labels.append( get_label( rxn, id_only=True ) )
            group_flows += fluxes[rxn.id]
        if group_flows/fluxes.sum()*100 >= max_group_flux_pct:
            groups.append( ('\n'.join( group_labels ), group_flows ))
            group_flows, group_labels = 0, []
    if group_flows > 0:
        groups.append( ( '\n'.join( group_labels ), group_flows ) ) 
    return zip(*groups)
            
def generate_googlechart_dataset( met, fluxes,
                                 min_group_flux_pct=10, 
                                 max_group_flux_pct=20,
                                cull_p=True):
    production_list = []
    for col in fluxes.columns:
        strain, ko, media = col
        producers, production_flux = get_producers( met, fluxes[col] )
        grouped_producer_labels, grouped_producer_flows = group_fluxes( producers,
                                                                      production_flux,
                                                                      min_group_flux_pct,
                                                                      max_group_flux_pct,
                                                                      cull_p=cull_p)
        production_list.extend(list(map(list, 
                       zip(  grouped_producer_labels,
                            ['{}[{}] in {} media with {} strain'.format( met.name, 
                                                   met.compartment, 
                                                   media, 
                                                 ko )
                            ]*len(grouped_producer_labels),
                            [10*flow for flow in grouped_producer_flows]))))
    return sorted(production_list, key=lambda x: (x[1], -np.abs(x[2])))
    
def plot_metabolite_flux( met, fluxes, titleFig, save='default', 
                         min_group_flux_pct=10, max_group_flux_pct=20 ):
    consumers, consumption_flux = get_consumers( met, fluxes )
    producers, production_flux = get_producers( met , fluxes )
    if len(consumers) == 0 or len(producers) == 0:
        return
    
    grouped_consumer_labels, grouped_consumer_flows = group_fluxes( consumers, consumption_flux, min_group_flux_pct, max_group_flux_pct )
    grouped_producer_labels, grouped_producer_flows = group_fluxes( producers, production_flux, min_group_flux_pct, max_group_flux_pct )
    labels= grouped_consumer_labels + grouped_producer_labels
             
    flows = [-flux for flux in grouped_consumer_flows] + list(grouped_producer_flows)
    orientations = []
    for i in range(len(grouped_consumer_labels)):
        if i < (len(grouped_consumer_labels)/2):
            orientations.append(-1)
        else:
            orientations.append(1)
    for j in range(len(grouped_producer_labels)):
        if j < len(grouped_producer_labels)/2:
            orientations.append(-1)
        else:
            orientations.append(1)
    idx_max = flows.index(max(flows))
    idx_min = flows.index(min(flows))
    orientations[idx_max] = 0
    orientations[idx_min] = 0
    
    pathlengths = [0.25]*len(labels)
    from matplotlib.sankey import Sankey
    matplotlib.rcParams.update({'font.size': 12})
    
    fig = figure(figsize=(18,18))
    title = '{}[{}] ({}) balances {}'.format(met.name, met.compartment, met.id, titleFig)
    ax = fig.add_subplot(1,1,1, xticks=[], yticks=[], title=title)
    sankey = Sankey(ax=ax, scale=0.15, offset=0.12)
    
    sankey.add( 
        labels = labels,
        flows = flows,
        orientations = orientations,
        pathlengths = pathlengths,
        patchlabel = '{}[{}]'.format(met.name,met.compartment),
        alpha=0.2, lw=2.0)
    diagrams = sankey.finish()
    diagrams[0].patch.set_facecolor('b')
    diagrams[0].text.set_fontweight('bold')
    diagrams[0].text.set_fontsize(25)
    
    matplotlib.rcParams.update({'font.size':20})
          
    if save != 'default':
        savefig(save)
def get_orientations_and_flows( consumer_labels, consumer_flows, producer_labels, producer_flows):
    flows = [-flux for flux in consumer_flows] + list(producer_flows)
    orientations = []
    for i in range(len(consumer_labels)):
            if i < (len(consumer_labels)/2):
                orientations.append(-1)
            else:
                orientations.append(1)
    for j in range(len(producer_labels)):
        if j < len(producer_labels)/2:
            orientations.append(-1)
        else:
            orientations.append(1)
    idx_max = flows.index(max(flows))
    idx_min = flows.index(min(flows))
    orientations[idx_max] = 0
    orientations[idx_min] = 0
    return orientations, flows

def multi_plot_metabolite_flux( met, fluxes,   
                         min_group_flux_pct=10, max_group_flux_pct=20 ):

    ax=gca()
    consumers, consumption_flux = get_consumers( met, fluxes )
    producers, production_flux = get_producers( met , fluxes )
    if len(consumers) == 0 or len(producers) == 0:
            return
    
    grouped_consumer_labels, grouped_consumer_flows = group_fluxes( consumers,
                                                                       consumption_flux, 
                                                                       min_group_flux_pct, 
                                                                       max_group_flux_pct 
                                                                      )
    grouped_producer_labels, grouped_producer_flows = group_fluxes( producers, 
                                                                       production_flux, 
                                                                       min_group_flux_pct, 
                                                                       max_group_flux_pct 
                                                                      )
    labels= grouped_consumer_labels + grouped_producer_labels
             
        
    orientations, flows = get_orientations_and_flows( grouped_consumer_labels,
                                                      grouped_consumer_flows,
                                                      grouped_producer_labels, 
                                                      grouped_producer_flows)
        
    
    #title_fig = '{}[{}] ({}) balances'.format(met.name, met.compartment, met.id)
    
    pathlengths = [0.25]*len(labels)
    from matplotlib.sankey import Sankey
    matplotlib.rcParams.update({'font.size': 12})
    
    #fig = figure(figsize=(18,18))
    #title = '{}[{}] ({}) balances {}'.format(met.name, met.compartment, met.id, titleFig)
    #ax = fig.add_subplot(1,1,1, xticks=[], yticks=[], title=title)
    sankey = Sankey(ax=ax, scale=0.15, offset=-0.05)
    
    sankey.add( 
        labels = labels,
        flows = flows,
        orientations = orientations,
        pathlengths = pathlengths,
        patchlabel = '{}[{}]'.format(met.name,met.compartment),
        trunklength=1.25,
        alpha=0.2, lw=2.0)
    diagrams = sankey.finish()
    diagrams[0].patch.set_facecolor('b')
    diagrams[0].text.set_fontweight('bold')
    diagrams[0].text.set_fontsize(25)
    
    matplotlib.rcParams.update({'font.size':24})
          
    
def metabolites_of_gene( model, gene ):
    metabolites = set()
    try:
        for rxn in model.genes.get_by_id(gene).reactions:
            metabolites |= set(rxn.reactants)
            metabolites |= set(rxn.products)
    except KeyError:
        pass
    return metabolites
def reactions_of_proteins( model, proteins ):
    reactions_of_proteins = set()
    for protein in proteins:
        try:
            reactions_of_proteins |= set(model.genes.get_by_id( protein ).reactions )
        except KeyError:
            pass
    return reactions_of_proteins

def metabolites_of_proteins( model, proteins ):
    metabolites_of_measured_proteins = set()
    for protein in proteins:
        metabolites_of_measured_proteins |= metabolites_of_gene( yarrowia, protein )
    return metabolites_of_measured_proteins

def all_reactions_of_metabolites( model, metabolites, reactions ):
    all_rxns_of_metabolites = set()
    for metabolite in metabolites:
        if len(metabolite.reactions) == len(metabolite.reactions & reactions):
            all_rxns_of_metabolites.add( metabolite )
    return all_rxns_of_metabolites



In [47]:
yli647 = cobra.io.read_sbml_model('iYLI647/iYLI647.sbmlv3_fbcv2.xml')
yli647.optimize()

,fluxes,reduced_costs
13BGH,0.000000,-5.294233e-02
13BGHe,0.000000,-2.775558e-17
ASADi,0.121794,0.000000e+00
2DDA7Ptm,0.000000,-0.000000e+00
2DHPtm,0.000000,0.000000e+00
...,...,...
FAO40p,0.000000,-2.775558e-17
LEUDH,2.019779,-3.469447e-18
MBCOAi,2.019779,1.301043e-18
MCCC,2.019779,1.170938e-17


In [59]:
mets = sorted([met for met in yli647.metabolites],key=lambda x: x.id)
rxns = sorted([rxn for rxn in yli647.reactions],key=lambda x: x.id)

tca_pwy = [rxn for rxn in yli647.reactions if rxn.subsystem =='Citric Acid Cycle']
tca_pwy

[<Reaction ACONTa at 0x119f5bd30>, <Reaction ACONTb at 0x119f5bb38>]

In [48]:
acon_C_m = cobra.Metabolite(
                            'acon_C_m',
                            formula = 'C6H3O6',
                            name = 'Cis-Aconitate',
                            charge = -3,
                            compartment='m')

acon_C_c = cobra.Metabolite('acon_C_c',
                            formula = 'C6H3O6',
                            name = 'Cis-Aconitate',
                            charge = -3,
                            compartment='c')

itacon_c = cobra.Metabolite('itacon_c',
                            formula='C5H4O4',
                            name='Itaconate',
                            charge=-2,
                            compartment='c')
itacon_e = cobra.Metabolite('itacon_e',
                            formula='C5H4O4',
                            name='Itaconate',
                            charge=-2,
                            compartment='e')
icit_c = yli647.metabolites.get_by_id('icit_c')
icit_m = yli647.metabolites.get_by_id('icit_m')

cit_c = yli647.metabolites.get_by_id('cit_c')
cit_m = yli647.metabolites.get_by_id('cit_m')
mal_L_c = yli647.metabolites.get_by_id('mal_L_c')
mal_L_m = yli647.metabolites.get_by_id('mal_L_m')
h2o_c = yli647.metabolites.get_by_id('h2o_c')
h2o_m = yli647.metabolites.get_by_id('h2o_m')
co2_c = yli647.metabolites.get_by_id('co2_c')
h_c = yli647.metabolites.get_by_id('h_c')

In [50]:
ACONT = yli647.reactions.get_by_id('ACONT')
ACONTm = yli647.reactions.get_by_id('ACONTm')
# ACONTa

ACONTa = cobra.Reaction('ACONTa',
                       name='Aconitase (half-reaction A, Citrate hydro-lase)',
                        subsystem='Citric Acid Cycle',
                        lower_bound=-100,
                        upper_bound=100,
                        objective_coefficient=0)
ACONTa.gene_reaction_rule = ACONT.gene_reaction_rule
ACONTa.add_metabolites({cit_c: -1.0,
                       acon_C_c: 1,
                       h2o_c: 1})

# ACONTb
ACONTb = cobra.Reaction('ACONTb',
                       name='Aconitase (half-reaction B, Isocitrate hydro-lase)',
                        subsystem='Citric Acid Cycle',
                        lower_bound=-100,
                        upper_bound=100,
                        objective_coefficient=0)
ACONTb.add_metabolites({acon_C_c: -1.0,
                       icit_c: 1,
                       h2o_c: -1})
ACONTb.gene_reaction_rule = ACONT.gene_reaction_rule

# ACONTam
ACONTam = cobra.Reaction('ACONTam',
                       name='Mitochondrial Aconitase (half-reaction A, Citrate hydro-lase)',
                        subsystem='Mitochondrial Citric Acid Cycle',
                        lower_bound=-100,
                        upper_bound=100,
                        objective_coefficient=0)
ACONTam.gene_reaction_rule = ACONTm.gene_reaction_rule
ACONTam.add_metabolites({cit_m: -1.0,
                       acon_C_m: 1,
                       h2o_m: 1})

# ACONTbm
ACONTbm = cobra.Reaction('ACONTbm',
                       name='Mitochondrial Aconitase (half-reaction B, Isocitrate hydro-lase)',
                        subsystem='Mitochondrial Citric Acid Cycle',
                        lower_bound=-100,
                        upper_bound=100,
                        objective_coefficient=0)
ACONTbm.gene_reaction_rule = ACONTm.gene_reaction_rule
ACONTbm.add_metabolites({ acon_C_m: -1.0,
                          icit_m:      1,
                          h2o_m:     -1})

# A
ACONTtam = cobra.Reaction('ACONTtam',
                        name='Cis-aconitate malate antiport, mitochondrial',
                         subsystem='Engineered Itaconate Biosynthesis',
                         lower_bound=-100,
                         upper_bound=100,
                         objective_coefficient=0)
ACONTtam.gene_reaction_rule='MttA'
ACONTtam.add_metabolites( {acon_C_m: -1,
                          mal_L_c: -1,
                          acon_C_c: 1,
                          mal_L_m: 1})


ITACON = cobra.Reaction('ITACON',
                       name='Aconitate Decarboxylase',
                       subsystem='Engineered Itaconate Biosynthesis',
                       lower_bound=-100,
                       upper_bound=100,
                       objective_coefficient=0)
ITACON.gene_reaction_rule='CAD1'
ITACON.add_metabolites({acon_C_c: -1,
                       h_c: -1,
                       co2_c: 1,
                       itacon_c: 1})

ITACONtr = cobra.Reaction('ITACONtr',
                         name='Itaconate transport',
                         subsystem='Engineered Itaconate Biosynthesis',
                          lower_bound=-100,
                          upper_bound=100,
                          objective_coefficient=0)
ITACONtr.add_metabolites({itacon_c: -1,
                         itacon_e: 1})
                          
EX_itacon_e = cobra.Reaction('EX_itacon_e',
                             name='Itaconate exchange',
                             subsystem='Engineered Itaconate Biosynthesis',
                             lower_bound=0,
                             upper_bound=100,
                             objective_coefficient=0)
EX_itacon_e.add_metabolites({itacon_e: -1})

yli647.add_reactions([ACONTa,
                      ACONTb,
                      ACONTam, 
                      ACONTbm, 
                      ACONTtam, 
                      ITACON, 
                      ITACONtr, 
                      EX_itacon_e])
yli647.reactions.ACONT.knock_out()

In [51]:
yli647.optimize()

,fluxes,reduced_costs
13BGH,0.000000,-5.294233e-02
13BGHe,0.000000,0.000000e+00
ASADi,0.121794,0.000000e+00
2DDA7Ptm,0.000000,-0.000000e+00
2DHPtm,0.000000,0.000000e+00
...,...,...
ACONTbm,0.000000,1.665335e-16
ACONTtam,0.000000,2.220446e-16
ITACON,0.000000,-0.000000e+00
ITACONtr,0.000000,-0.000000e+00


In [52]:
jdz649 = yli647.copy()
cobra.io.save_json_model(jdz649,'iYLI647/jdz649.itaconate.json')


In [53]:
jdz649 = cobra.io.load_json_model('iYLI647/jdz649.itaconate.json')
jdz649.optimize()

,fluxes,reduced_costs
13BGH,0.000000,-5.294233e-02
13BGHe,0.000000,0.000000e+00
ASADi,0.121794,2.775558e-17
2DDA7Ptm,0.000000,-0.000000e+00
2DHPtm,0.000000,0.000000e+00
...,...,...
ACONTbm,0.000000,0.000000e+00
ACONTtam,0.000000,1.387779e-17
ITACON,0.000000,1.301043e-17
ITACONtr,0.000000,-0.000000e+00


In [66]:
with jdz649:
    jdz649.reactions.CITtam.knock_out()
    soln = jdz649.optimize()
    display(soln)

,fluxes,reduced_costs
13BGH,0.000000e+00,-5.574913e-02
13BGHe,0.000000e+00,0.000000e+00
ASADi,4.595958e+00,-2.220446e-16
2DDA7Ptm,2.723453e-15,-0.000000e+00
2DHPtm,0.000000e+00,-0.000000e+00
...,...,...
ACONTbm,0.000000e+00,0.000000e+00
ACONTtam,1.114899e+01,0.000000e+00
ITACON,1.114899e+01,5.117434e-17
ITACONtr,1.114899e+01,0.000000e+00


In [54]:

jdz649.objective= "ITACONtr"
jdz649.optimize()

,fluxes,reduced_costs
13BGH,0.000000,-1.114983e-01
13BGHe,0.000000,0.000000e+00
ASADi,4.595958,0.000000e+00
2DDA7Ptm,0.000000,-0.000000e+00
2DHPtm,0.000000,0.000000e+00
...,...,...
ACONTbm,0.000000,0.000000e+00
ACONTtam,0.000000,0.000000e+00
ITACON,11.148990,-4.336809e-17
ITACONtr,11.148990,0.000000e+00


In [61]:
subsystems = set([rxn.subsystem for rxn in jdz649.reactions])
print('\n'.join(subsystems))


Citric Acid Cycle
Mitochondrial Citric Acid Cycle
Engineered Itaconate Biosynthesis


In [84]:
TCA_m_pwy = set()

In [86]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('CSm'),jdz649.reactions.get_by_id('ACONTam')}
jdz649.metabolites.cit_m.summary()

PRODUCING REACTIONS -- Citrate (cit_m)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%    11.1  CSm       accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m

CONSUMING REACTIONS -- Citrate (cit_m)
--------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%    11.1  ACONTam   cit_m <=> acon_C_m + h2o_m


In [87]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('PYRt2m'),jdz649.reactions.get_by_id('PDHm')}
jdz649.metabolites.pyr_m.summary()

PRODUCING REACTIONS -- Pyruvate (pyr_m)
---------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%    6.55  PYRt2m    h_c + pyr_c <=> h_m + pyr_m

CONSUMING REACTIONS -- Pyruvate (pyr_m)
---------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%    6.55  PDHm      coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m


In [89]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('ACSm'),jdz649.reactions.get_by_id('CSm')}
jdz649.metabolites.accoa_m.summary()

PRODUCING REACTIONS -- Acetyl_CoA (accoa_m)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
59%     6.55  PDHm      coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m
41%     4.6   ACSm      ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m

CONSUMING REACTIONS -- Acetyl_CoA (accoa_m)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%   11.1   CSm       accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m


In [90]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('ACtm'),jdz649.reactions.get_by_id('ACSm')}
jdz649.metabolites.ac_m.summary()

PRODUCING REACTIONS -- Acetate (ac_m)
-------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ------------------------------------------------
100%     4.6  ACtm      ac_c <=> ac_m

CONSUMING REACTIONS -- Acetate (ac_m)
-------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ------------------------------------------------
100%     4.6  ACSm      ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m


In [80]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('CSm'),jdz649.reactions.get_by_id('ACONTam')}
jdz649.metabolites.coa_m.summary()

PRODUCING REACTIONS -- Coenzyme_A (coa_m)
-----------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%   11.1   CSm       accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m

CONSUMING REACTIONS -- Coenzyme_A (coa_m)
-----------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
59%     6.55  PDHm      coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m
41%     4.6   ACSm      ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m


In [91]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('OAAt2m')} 

jdz649.metabolites.oaa_m.summary()


PRODUCING REACTIONS -- Oxaloacetate (oaa_m)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%    11.1  OAAt2m    h_c + oaa_c <=> h_m + oaa_m

CONSUMING REACTIONS -- Oxaloacetate (oaa_m)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------------
100%    11.1  CSm       accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m


In [93]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('ACONTtam'),
              jdz649.reactions.get_by_id('ACONTam')}
jdz649.metabolites.acon_C_m.summary()

PRODUCING REACTIONS -- Cis-Aconitate (acon_C_m)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
100%    11.1  ACONTam   cit_m <=> acon_C_m + h2o_m

CONSUMING REACTIONS -- Cis-Aconitate (acon_C_m)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
100%    11.1  ACONTtam  acon_C_m + mal_L_c <=> acon_C_c + mal_L_m


In [95]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('ACONTtam'),
              jdz649.reactions.get_by_id('ITACON')}
jdz649.metabolites.acon_C_c.summary()

PRODUCING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
100%    11.1  ACONTtam  acon_C_m + mal_L_c <=> acon_C_c + mal_L_m

CONSUMING REACTIONS -- Cis-Aconitate (acon_C_c)
-----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------------
100%    11.1  ITACON    acon_C_c + h_c <=> co2_c + itacon_c


In [97]:
TCA_m_pwy |= {jdz649.reactions.get_by_id('ITACONtr')}
jdz649.metabolites.itacon_c.summary()

PRODUCING REACTIONS -- Itaconate (itacon_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------
100%    11.1  ITACON    acon_C_c + h_c <=> co2_c + itacon_c

CONSUMING REACTIONS -- Itaconate (itacon_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -----------------------------------
100%    11.1  ITACONtr  itacon_c <=> itacon_e


In [101]:
jdz649.reactions.get_by_id('ITACONtr')

Reaction identifier,ITACONtr
Name,Itaconate transport
Memory address,0x011d03dfd0
Stoichiometry,itacon_c <=> itacon_e Itaconate <=> Itaconate
GPR,
Lower bound,-100
Upper bound,100


In [102]:
soln

,fluxes,reduced_costs
13BGH,0.000000e+00,-5.574913e-02
13BGHe,0.000000e+00,0.000000e+00
ASADi,4.595958e+00,-2.220446e-16
2DDA7Ptm,2.723453e-15,-0.000000e+00
2DHPtm,0.000000e+00,-0.000000e+00
...,...,...
ACONTbm,0.000000e+00,0.000000e+00
ACONTtam,1.114899e+01,0.000000e+00
ITACON,1.114899e+01,5.117434e-17
ITACONtr,1.114899e+01,0.000000e+00


In [115]:
itacon_pwy = cobra.Model('Itaconate_pathway')
itacon_pwy.add_reactions(TCA_m_pwy)
cobra.io.save_json_model(itacon_pwy, 
                         'Itaconate_pathway.coords.json',
                         pretty=True)

In [116]:
fluxes = soln.to_frame()['fluxes']
display(flux_map(itacon_pwy,
                display_name_format=lambda x: x.id,
                 excluded_metabolites=['h_c','h_m','h2o_m','atp_m','amp_m','ppi_m', 'nad_m','nadh_m','coa_m'],
                flux_dict={rxn.id: fluxes[rxn.id]                           
                                for rxn in itacon_pwy.reactions}))
                